# VIX-Style Volatility Index - Demo

This notebook demonstrates:
1. VIX calculation from options data
2. Variance term structure visualization
3. Variance swap pricing
4. Forward volatility extraction

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.vix_index import vix_index, build_qk, compute_forward_and_k0
from src.pricing import (
    variance_swap_fair_strike,
    variance_swap_pnl,
    vix_forward,
    vol_swap_convexity_adjustment,
)

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Load Data

In [ ]:
# Load data files
options = pd.read_csv("data/henry hub european options.csv")
rates = pd.read_csv("data/treasury yield curve rates.csv")
calendar = pd.read_csv("data/cme holidays.csv")

# Convert dates
options["futures-expirationDate"] = pd.to_datetime(options["futures-expirationDate"])
options["tradeDate"] = pd.to_datetime(options["tradeDate"])
options["futures-updated"] = pd.to_datetime(options["futures-updated"])
options["options-updated"] = pd.to_datetime(options["options-updated"])
rates["Date"] = pd.to_datetime(rates["Date"])

print(f"Options: {len(options):,} rows")
print(f"Date range: {options['tradeDate'].min()} to {options['tradeDate'].max()}")

## 2. Calculate VIX

In [ ]:
# Calculate VIX for a specific date
result = vix_index(
    options_df=options,
    rates_df=rates,
    calendar_df=calendar,
    options_id=1352,
    trade_date="2020-11-12",
    front_months=2,
    rear_months=4,
    target_days=90,
    price_scale=100,
)

print(f"VIX Index: {result['vix']:.2f}%")
print(f"\nNear-term (T1={result['T1']*365:.0f}d): σ² = {result['sigma2_1']:.4f} → σ = {np.sqrt(result['sigma2_1'])*100:.2f}%")
print(f"Far-term  (T2={result['T2']*365:.0f}d): σ² = {result['sigma2_2']:.4f} → σ = {np.sqrt(result['sigma2_2'])*100:.2f}%")

## 3. Variance Term Structure

In [ ]:
# Calculate VIX for multiple maturities
maturities = [(1, 2), (2, 3), (2, 4), (3, 4), (3, 5), (4, 5)]
target_days_list = [30, 45, 90, 90, 120, 120]

term_structure = []

for (front, rear), target in zip(maturities, target_days_list):
    try:
        res = vix_index(
            options_df=options,
            rates_df=rates,
            calendar_df=calendar,
            options_id=1352,
            trade_date="2020-11-12",
            front_months=front,
            rear_months=rear,
            target_days=target,
            price_scale=100,
        )
        term_structure.append({
            'target_days': target,
            'vix': res['vix'],
            'sigma2': (res['sigma2_1'] + res['sigma2_2']) / 2,
            'T_avg': (res['T1'] + res['T2']) / 2,
        })
    except ValueError as e:
        print(f"Skipped {front}M/{rear}M target={target}d: {e}")

df_term = pd.DataFrame(term_structure).drop_duplicates(subset='target_days')
df_term = df_term.sort_values('target_days')
print(df_term)

In [ ]:
# Plot term structure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Volatility term structure
ax1.plot(df_term['target_days'], df_term['vix'], 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Target Maturity (days)', fontsize=12)
ax1.set_ylabel('Implied Volatility (%)', fontsize=12)
ax1.set_title('Volatility Term Structure', fontsize=14)
ax1.grid(True, alpha=0.3)

# Variance term structure (total variance)
ax2.plot(df_term['target_days'], df_term['sigma2'] * df_term['target_days']/365, 'ro-', linewidth=2, markersize=8)
ax2.set_xlabel('Target Maturity (days)', fontsize=12)
ax2.set_ylabel('Total Variance (σ² × T)', fontsize=12)
ax2.set_title('Total Variance Term Structure', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('term_structure.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Option Smile (Q(K) Distribution)

In [ ]:
# Get options for a specific expiration
trade_date = pd.Timestamp("2020-11-12")
mask = (options["options-id"] == 1352) & (options["tradeDate"] == trade_date)
opts = options.loc[mask].copy()

# Get unique expirations
expirations = opts["futures-expirationDate"].unique()
print(f"Available expirations: {sorted(expirations)}")

In [ ]:
# Build Q(K) for one expiration
exp_date = pd.Timestamp("2021-01-01")
opts_exp = opts[opts["futures-expirationDate"] == exp_date].copy()

if not opts_exp.empty:
    # Compute forward and K0
    F, K0 = compute_forward_and_k0(opts_exp, r=0.001, T=0.15)
    
    # Build Q(K)
    qk = build_qk(opts_exp, K0)
    
    # Scale prices
    qk_scaled = qk * 100  # price_scale
    
    print(f"Forward: {F:.2f}, K0: {K0:.2f}")
    print(f"Strikes used: {len(qk)}")

In [ ]:
# Plot Q(K) distribution
fig, ax = plt.subplots(figsize=(12, 6))

strikes = qk_scaled.index.values
prices = qk_scaled.values

# Color by moneyness
colors = ['red' if k < K0 else 'blue' if k > K0 else 'green' for k in strikes]

ax.bar(strikes, prices, color=colors, alpha=0.7, width=(strikes[1]-strikes[0])*0.8)
ax.axvline(K0, color='black', linestyle='--', linewidth=2, label=f'K₀ = {K0:.0f}')
ax.axvline(F, color='gray', linestyle=':', linewidth=2, label=f'F = {F:.0f}')

ax.set_xlabel('Strike (K)', fontsize=12)
ax.set_ylabel('Q(K) - Option Price (scaled)', fontsize=12)
ax.set_title('Option Price Distribution Q(K)\n(Red=OTM Puts, Blue=OTM Calls, Green=ATM)', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('qk_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Variance Swap Pricing

In [ ]:
# Variance swap parameters
sigma2 = result['sigma2_1']  # Use near-term variance
notional_vega = 100_000  # $100k vega notional

# Fair variance strike
k_var = variance_swap_fair_strike(sigma2)
k_vol = np.sqrt(k_var) * 100  # Convert to %

print("=" * 50)
print("VARIANCE SWAP PRICING")
print("=" * 50)
print(f"Fair Variance Strike (K_var): {k_var:.6f}")
print(f"Fair Volatility Strike (K_vol): {k_vol:.2f}%")
print(f"Vega Notional: ${notional_vega:,.0f}")

In [ ]:
# P&L scenarios at maturity
realized_vols = np.linspace(10, 40, 50)  # Realized vol scenarios (%)
realized_vars = (realized_vols / 100) ** 2  # Convert to variance

pnls = [variance_swap_pnl(k_var, rv, notional_vega) for rv in realized_vars]

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(realized_vols, [p/1000 for p in pnls], 'b-', linewidth=2)
ax.axhline(0, color='black', linewidth=0.5)
ax.axvline(k_vol, color='red', linestyle='--', linewidth=2, label=f'Strike = {k_vol:.1f}%')

ax.fill_between(realized_vols, [p/1000 for p in pnls], 0, 
                where=[p > 0 for p in pnls], alpha=0.3, color='green', label='Profit')
ax.fill_between(realized_vols, [p/1000 for p in pnls], 0, 
                where=[p < 0 for p in pnls], alpha=0.3, color='red', label='Loss')

ax.set_xlabel('Realized Volatility (%)', fontsize=12)
ax.set_ylabel('P&L ($k)', fontsize=12)
ax.set_title(f'Variance Swap P&L at Maturity\n(Vega Notional = ${notional_vega:,.0f})', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('variance_swap_pnl.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Forward Volatility

In [ ]:
# Extract forward volatility
T1, T2 = result['T1'], result['T2']
sigma2_1, sigma2_2 = result['sigma2_1'], result['sigma2_2']

# Forward vol from T1 to T2
fwd_vix = vix_forward(sigma2_1, sigma2_2, T1, T2, T1, T2)

print("=" * 50)
print("FORWARD VOLATILITY")
print("=" * 50)
print(f"Spot Vol (T1={T1*365:.0f}d): {np.sqrt(sigma2_1)*100:.2f}%")
print(f"Spot Vol (T2={T2*365:.0f}d): {np.sqrt(sigma2_2)*100:.2f}%")
print(f"Forward Vol ({T1*365:.0f}d → {T2*365:.0f}d): {fwd_vix:.2f}%")

In [ ]:
# Vol swap convexity adjustment
vol_swap_strike = vol_swap_convexity_adjustment(sigma2_1, vol_of_vol=0.5)
var_swap_vol = np.sqrt(sigma2_1)

print("\n" + "=" * 50)
print("VOL SWAP vs VAR SWAP")
print("=" * 50)
print(f"Variance Swap Strike (√K_var): {var_swap_vol*100:.2f}%")
print(f"Volatility Swap Strike (approx): {vol_swap_strike*100:.2f}%")
print(f"Convexity Adjustment: {(var_swap_vol - vol_swap_strike)*100:.2f}%")

## 7. Summary

In [ ]:
print("\n" + "=" * 60)
print("SUMMARY - VIX CALCULATION RESULTS")
print("=" * 60)
print(f"Trade Date: 2020-11-12")
print(f"Target Maturity: 90 days")
print(f"\nVIX Index: {result['vix']:.2f}%")
print(f"\nNear-term Contract:")
print(f"  Expiry: {result['T1']*365:.0f} days")
print(f"  Forward: {result['F1']:.2f}")
print(f"  ATM Strike: {result['K0_1']:.2f}")
print(f"  Implied Vol: {np.sqrt(result['sigma2_1'])*100:.2f}%")
print(f"\nFar-term Contract:")
print(f"  Expiry: {result['T2']*365:.0f} days")
print(f"  Forward: {result['F2']:.2f}")
print(f"  ATM Strike: {result['K0_2']:.2f}")
print(f"  Implied Vol: {np.sqrt(result['sigma2_2'])*100:.2f}%")
print(f"\nVariance Swap Fair Strike: {k_vol:.2f}%")
print(f"Forward Volatility ({result['T1']*365:.0f}d→{result['T2']*365:.0f}d): {fwd_vix:.2f}%")
print("=" * 60)